In [1]:
import os 
import numpy as np
import cv2
from matplotlib import pyplot as plt
import skimage


# Importing original data to kaggle's working directory

In [2]:
import shutil

# Path to your 'original' folder (read-only)
original_folder = '/kaggle/input/watershed/Final data/Final data/New folder/Original'

# Path to a writable directory
working_folder = '/kaggle/working/original_renamed/'

# Create the writable directory if it doesn't exist
if not os.path.exists(working_folder):
    os.makedirs(working_folder)

# List all the image files in the 'original' folder
image_files = sorted(os.listdir(original_folder))

# Loop through the files, copy to writable directory, and rename
for i, filename in enumerate(image_files):
    # Get the file extension (e.g., .jpg or .png)
    file_extension = os.path.splitext(filename)[1]
    
    # Create a new filename (e.g., img_0001.png, img_0002.png, etc.)
    new_filename = f"{i+1:04d}{file_extension}"
    
    # Full path to the current file
    current_path = os.path.join(original_folder, filename)
    
    # Full path to the new file in the writable directory
    new_path = os.path.join(working_folder, new_filename)
    
    # Copy and rename the file to the writable folder
    shutil.copy(current_path, new_path)

print("Renaming and copying complete.")

Renaming and copying complete.


# Importing Mask data to kaggle's working directory

In [3]:
import shutil

# Path to your 'mask' folder (read-only)
mask_folder = '/kaggle/input/watershed/Final data/Final data/New folder/Mask'

# Path to a writable directory for renamed masks
mask_working_folder = '/kaggle/working/mask_renamed/'

# Create the writable directory if it doesn't exist
if not os.path.exists(mask_working_folder):
    os.makedirs(mask_working_folder)

# List all the mask image files in the 'mask' folder
mask_files = sorted(os.listdir(mask_folder))

# Loop through the files, copy to writable directory, and rename
for i, filename in enumerate(mask_files):
    # Get the file extension (e.g., .jpg or .png)
    file_extension = os.path.splitext(filename)[1]
    
    # Create a new filename (e.g., mask_0001.png, mask_0002.png, etc.)
    new_filename = f"{i+1:04d}{file_extension}"
    
    # Full path to the current file
    current_mask_path = os.path.join(mask_folder, filename)
    
    # Full path to the new file in the writable directory
    new_mask_path = os.path.join(mask_working_folder, new_filename)
    
    # Copy and rename the file to the writable folder
    shutil.copy(current_mask_path, new_mask_path)

print("Renaming and copying of mask images complete.")

Renaming and copying of mask images complete.


# setting directory variables

In [4]:
image_dir = '/kaggle/working/original_renamed'
output_dir = '/kaggle/working/watershed_segmented_mask'

# Sorting all the images 

In [5]:
def get_numeric_value(filename):
    return int(''.join(filter(str.isdigit, filename)))  # Extract digits and convert to an integer

# Get all '.png' files in the directory
image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]

# Sort files numerically based on the extracted numeric value
image_files_sorted = sorted(image_files, key=get_numeric_value)

print(image_files_sorted)


['0001.png', '0002.png', '0003.png', '0004.png', '0005.png', '0006.png', '0007.png', '0009.png', '0010.png', '0011.png', '0012.png', '0013.png', '0014.png', '0015.png', '0016.png', '0018.png', '0019.png', '0020.png', '0021.png', '0022.png', '0024.png', '0025.png', '0026.png', '0027.png', '0028.png', '0029.png', '0030.png', '0031.png', '0032.png', '0033.png', '0034.png', '0035.png', '0036.png', '0037.png', '0038.png', '0039.png', '0040.png', '0041.png', '0042.png', '0043.png', '0045.png', '0046.png', '0047.png', '0048.png', '0049.png', '0050.png', '0051.png', '0052.png', '0053.png', '0056.png', '0058.png', '0059.png', '0060.png', '0061.png', '0062.png', '0063.png', '0064.png', '0066.png', '0067.png', '0068.png', '0069.png', '0070.png', '0071.png', '0072.png', '0073.png', '0074.png', '0075.png', '0076.png', '0077.png', '0078.png', '0079.png', '0080.png', '0081.png', '0083.png', '0084.png', '0085.png', '0086.png', '0087.png', '0088.png', '0089.png', '0090.png', '0091.png', '0092.png', '00

# Iterating over all the images in the dataset

In [6]:
for image_file in image_files:
    img_path = os.path.join(image_dir, image_file)

# Reading the image

In [7]:
src = cv2.imread(img_path)
img = src.copy()
gray = img[:,:,0]

# Applying watershed algorithm for segmenting all the original images and saving them into a new directory

In [8]:
from skimage import io

# Define the input directory containing the images
input_dir = '/kaggle/working/original_renamed'  # Replace with your directory path

# Define the output directory where you want to save segmented images
output_dir = '/kaggle/working/watershed_segmented_mask'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)  # Create the output directory if it doesn't exist

# List all image files in the directory (assuming they have extensions like .png, .jpg)
image_files = [f for f in os.listdir(input_dir) if f.endswith(('.png'))]

# Check if any files are found
if not image_files:
    print("No image files found in the directory!")
else:
    print(f"Found {len(image_files)} image files.")

# Iterate over all images in the directory
for image_file in image_files:
    # Load the image
    img_path = os.path.join(input_dir, image_file)
    img = cv2.imread(img_path)

    # Check if the image was loaded correctly
    if img is None:
        print(f"Error loading {image_file}")
        continue

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply thresholding and morphological operations
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=3)
    sure_bg = cv2.dilate(opening, kernel, iterations=9)

    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 3)
    ret, sure_fg = cv2.threshold(dist_transform, 0.5 * dist_transform.max(), 255, 0)

    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    ret, markers1 = cv2.connectedComponents(sure_fg)
    markers = markers1 + 10
    markers[unknown == 255] = 0
    markers3 = cv2.watershed(img, markers)
    img[markers3 == -1] = [255, 255, 255]

    # Convert the segmented image to grayscale
    gr = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply thresholding
    ret1, th1 = cv2.threshold(gr, 127, 255, 0)
    imagem = cv2.bitwise_not(th1)

    # Define the output path for the segmented image
    output_path = os.path.join(output_dir, image_file)
    
    # Save the segmented image
    cv2.imwrite(output_path, imagem)
    print(f"Segmented image saved: {output_path}")

# Now, display all saved segmented images
# segmented_image_files = [f for f in os.listdir(output_dir) if f.endswith('.png')]
# for segmented_image_file in segmented_image_files:
#     segmented_image_path = os.path.join(output_dir, segmented_image_file)
#     segmented_image = cv2.imread(segmented_image_path)

#     # Display the image using skimage.io.imshow
#     io.imshow(segmented_image)
#     plt.title(segmented_image_file)
#     plt.axis('off')  # Hide axis
#     plt.show()  # Display the image



Found 1209 image files.
Segmented image saved: /kaggle/working/watershed_segmented_mask/0862.png
Segmented image saved: /kaggle/working/watershed_segmented_mask/0882.png
Segmented image saved: /kaggle/working/watershed_segmented_mask/0986.png
Segmented image saved: /kaggle/working/watershed_segmented_mask/0814.png
Segmented image saved: /kaggle/working/watershed_segmented_mask/0081.png
Segmented image saved: /kaggle/working/watershed_segmented_mask/0558.png
Segmented image saved: /kaggle/working/watershed_segmented_mask/0224.png
Segmented image saved: /kaggle/working/watershed_segmented_mask/0747.png
Segmented image saved: /kaggle/working/watershed_segmented_mask/0833.png
Segmented image saved: /kaggle/working/watershed_segmented_mask/1123.png
Segmented image saved: /kaggle/working/watershed_segmented_mask/0486.png
Segmented image saved: /kaggle/working/watershed_segmented_mask/0768.png
Segmented image saved: /kaggle/working/watershed_segmented_mask/0901.png
Segmented image saved: /kag

# Applying Jaccard and Dice-coefficient indexes

In [9]:
import tensorflow as tf
from tensorflow.keras import backend as K

# Define directories
original_masks_dir = '/kaggle/working/mask_renamed'
segmented_masks_dir = '/kaggle/working/watershed_segmented_mask'

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return dice

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def jaccard_index(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f + y_pred_f) - intersection
    iou = (intersection + smooth) / (union + smooth)
    return iou

# List files in both directories
original_mask_files = [f for f in os.listdir(original_masks_dir) if f.endswith('.png')]
segmented_mask_files = [f for f in os.listdir(segmented_masks_dir) if f.endswith('.png')]

# Initialize metrics lists
dice_scores = []
jaccard_scores = []

# Iterate through the mask files
for mask_file in original_mask_files:
    if mask_file in segmented_mask_files:
        # Load original and segmented images
        original_mask_path = os.path.join(original_masks_dir, mask_file)
        segmented_mask_path = os.path.join(segmented_masks_dir, mask_file)
        
        original_mask = cv2.imread(original_mask_path, cv2.IMREAD_GRAYSCALE)
        segmented_mask = cv2.imread(segmented_mask_path, cv2.IMREAD_GRAYSCALE)

        # Binarize the masks (optional depending on your mask format)
        original_mask = (original_mask > 0).astype(np.float32)
        segmented_mask = (segmented_mask > 0).astype(np.float32)

        # Convert to tensors
        y_true = tf.convert_to_tensor(original_mask)
        y_pred = tf.convert_to_tensor(segmented_mask)

        # Calculate metrics
        dice_score = dice_coef(y_true, y_pred).numpy()  # Convert tensor to numpy for display
        jaccard_score = jaccard_index(y_true, y_pred).numpy()  # Convert tensor to numpy for display
        
        dice_scores.append(dice_score)
        jaccard_scores.append(jaccard_score)
        
        print(f"File: {mask_file}, Dice Coef: {dice_score}, Jaccard Index: {jaccard_score}")

# Optionally, calculate the average scores
average_dice = np.mean(dice_scores)
average_jaccard = np.mean(jaccard_scores)
print(f"Average Dice Coefficient: {average_dice}")
print(f"Average Jaccard Index: {average_jaccard}")

File: 0862.png, Dice Coef: 0.5283471345901489, Jaccard Index: 0.3590167462825775
File: 0882.png, Dice Coef: 0.4630442261695862, Jaccard Index: 0.3012746572494507
File: 0986.png, Dice Coef: 0.9094681143760681, Jaccard Index: 0.8339676856994629
File: 0814.png, Dice Coef: 0.9293028712272644, Jaccard Index: 0.8679419755935669
File: 0081.png, Dice Coef: 0.884781002998352, Jaccard Index: 0.7933698892593384


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


File: 0558.png, Dice Coef: 0.3923432528972626, Jaccard Index: 0.24404793977737427
File: 0224.png, Dice Coef: 0.8425989151000977, Jaccard Index: 0.7280097603797913
File: 0747.png, Dice Coef: 0.9100415706634521, Jaccard Index: 0.8349325060844421
File: 0833.png, Dice Coef: 0.8854836821556091, Jaccard Index: 0.7945005893707275
File: 1123.png, Dice Coef: 0.8863295912742615, Jaccard Index: 0.7958636283874512
File: 0486.png, Dice Coef: 0.7770901322364807, Jaccard Index: 0.6354438066482544
File: 0768.png, Dice Coef: 0.9277433753013611, Jaccard Index: 0.8652253746986389
File: 0901.png, Dice Coef: 0.9203172326087952, Jaccard Index: 0.8523961305618286
File: 0600.png, Dice Coef: 0.253711462020874, Jaccard Index: 0.14528676867485046
File: 1252.png, Dice Coef: 0.8074247241020203, Jaccard Index: 0.6770433187484741
File: 1083.png, Dice Coef: 0.926156759262085, Jaccard Index: 0.8624693155288696
File: 0713.png, Dice Coef: 0.9418115019798279, Jaccard Index: 0.8900225162506104


libpng warning: iCCP: known incorrect sRGB profile


File: 0438.png, Dice Coef: 0.9178659319877625, Jaccard Index: 0.8481999039649963
File: 0352.png, Dice Coef: 0.856574296951294, Jaccard Index: 0.7491300702095032
File: 1154.png, Dice Coef: 0.844271719455719, Jaccard Index: 0.7305108308792114
File: 0683.png, Dice Coef: 0.9029275178909302, Jaccard Index: 0.8230336904525757
File: 0921.png, Dice Coef: 0.8039061427116394, Jaccard Index: 0.6721097230911255
File: 1010.png, Dice Coef: 0.8888350129127502, Jaccard Index: 0.7999128103256226


...

File: 0321.png, Dice Coef: 0.9014121890068054, Jaccard Index: 0.8205191493034363
File: 0210.png, Dice Coef: 0.23518994450569153, Jaccard Index: 0.1332673579454422
File: 0675.png, Dice Coef: 0.27836480736732483, Jaccard Index: 0.1616870015859604
File: 0922.png, Dice Coef: 0.7347793579101562, Jaccard Index: 0.5807523727416992
File: 0703.png, Dice Coef: 0.9487686157226562, Jaccard Index: 0.90253084897995
File: 0943.png, Dice Coef: 0.35085853934288025, Jaccard Index: 0.21275316178798676


libpng warning: iCCP: known incorrect sRGB profile


File: 0254.png, Dice Coef: 0.8510168194770813, Jaccard Index: 0.740669846534729
File: 1259.png, Dice Coef: 0.8727319240570068, Jaccard Index: 0.7742010354995728
File: 1221.png, Dice Coef: 0.8418776392936707, Jaccard Index: 0.7269333004951477
File: 0679.png, Dice Coef: 0.7928770780563354, Jaccard Index: 0.6568325161933899
File: 0415.png, Dice Coef: 0.9178751707077026, Jaccard Index: 0.8482156991958618
File: 0385.png, Dice Coef: 0.7820804119110107, Jaccard Index: 0.6421447992324829


libpng warning: iCCP: known incorrect sRGB profile


File: 0757.png, Dice Coef: 0.04002487659454346, Jaccard Index: 0.02042231895029545
File: 1170.png, Dice Coef: 0.20220468938350677, Jaccard Index: 0.11247498542070389
File: 0405.png, Dice Coef: 0.4940992295742035, Jaccard Index: 0.3281095325946808
File: 0517.png, Dice Coef: 0.8525258898735046, Jaccard Index: 0.7429589033126831
File: 0207.png, Dice Coef: 0.7892915606498718, Jaccard Index: 0.6519256234169006
Average Dice Coefficient: 0.7780851125717163
Average Jaccard Index: 0.676308810710907
